<a href="https://colab.research.google.com/github/kaisarmasum/Generative-AI-Use-Case-Summarize-Dialogue/blob/main/VQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import requests
from PIL import Image, ImageDraw, ImageFont
from transformers import ViltProcessor, ViltForQuestionAnswering
import torch

# Load a font
try:
    font = ImageFont.truetype("arial.ttf", 24)
except IOError:
    font = ImageFont.load_default()

# Load the ViLT processor and model
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# Define the text query
text = "who many?"

# List of image paths with their corresponding start time, end time, and duration
images = [
    { "path": "/content/cat.jpeg", "start_time": "00:00:01", "end_time": "00:00:02", "duration": "00:00:01" },
    { "path": "/content/dog.jpg", "start_time": "00:00:03", "end_time": "00:00:04", "duration": "00:00:01" }
    # Add more image paths and their details here
]

for idx, image_info in enumerate(images, start=1):
    # Load an input image
    image = Image.open(image_info["path"])

    # Extract image details
    start_time = image_info["start_time"]
    end_time = image_info["end_time"]
    duration = image_info["duration"]

    # Add frame number, timeframe, and duration
    frame_number = idx  # Example frame number

    # Draw the frame number, timeframe, and duration on the image
    draw = ImageDraw.Draw(image)
    text_frame = f"Frame: {frame_number}"
    text_timeframe = f"Timeframe: {start_time} - {end_time}"
    text_duration = f"Duration: {duration}"

    # Set text positions
    text_position_frame = (10, 10)
    text_position_timeframe = (10, 40)
    text_position_duration = (10, 70)

    # Draw text on the image
    draw.text(text_position_frame, text_frame, fill="white", font=font)
    draw.text(text_position_timeframe, text_timeframe, fill="white", font=font)
    draw.text(text_position_duration, text_duration, fill="white", font=font)

    # Prepare inputs
    inputs = processor(image, text, return_tensors="pt")

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    idx = logits.argmax(-1).item()
    predicted_answer = model.config.id2label[idx]

    # Add the predicted answer to the image
    text_predicted_answer = f"Predicted Answer: {predicted_answer}"
    text_position_predicted_answer = (10, 100)
    draw.text(text_position_predicted_answer, text_predicted_answer, fill="white", font=font)

    # Display the image (optional)
    image.show()

    # Save the image with annotations (optional)
    image.save(f"annotated_image_{idx}.jpg")

    print(f"Image {idx}:")
    print(f"Predicted answer: {predicted_answer}")
    print(f"Timeframe: {start_time} - {end_time}")
    print(f"Duration: {duration}")

Image 12:
Predicted answer: 1
Timeframe: 00:00:01 - 00:00:02
Duration: 00:00:01
Image 12:
Predicted answer: 1
Timeframe: 00:00:03 - 00:00:04
Duration: 00:00:01
